In [1]:
# ✅ Setup — install & import everything we need
# Run this cell first!

import numpy as np
import os
import time

# Check if we're in Colab for the interactive display
try:
    from IPython.display import display, HTML, clear_output
    IN_NOTEBOOK = True
except ImportError:
    IN_NOTEBOOK = False

print("Setup complete! Ready to play 🎮")


Setup complete! Ready to play 🎮


In [2]:
# Game constants
ROWS, COLS = 6, 7
EMPTY, PLAYER, AI = 0, 1, 2

def create_board():
    """Create an empty 6x7 board."""
    return np.zeros((ROWS, COLS), dtype=int)

def get_valid_columns(board):
    """Return list of columns that still have space."""
    return [c for c in range(COLS) if board[0][c] == EMPTY]

def drop_piece(board, col, piece):
    """Drop a piece into a column. Returns new board (or None if column is full)."""
    b = board.copy()
    for r in range(ROWS - 1, -1, -1):
        if b[r][col] == EMPTY:
            b[r][col] = piece
            return b
    return None

def check_win(board, piece):
    """Check if 'piece' has 4 in a row (horizontal, vertical, or diagonal)."""
    for r in range(ROWS):
        for c in range(COLS):
            # Horizontal →
            if c + 3 < COLS and all(board[r][c+i] == piece for i in range(4)):
                return True
            # Vertical ↓
            if r + 3 < ROWS and all(board[r+i][c] == piece for i in range(4)):
                return True
            # Diagonal ↘
            if r + 3 < ROWS and c + 3 < COLS and all(board[r+i][c+i] == piece for i in range(4)):
                return True
            # Diagonal ↙
            if r + 3 < ROWS and c - 3 >= 0 and all(board[r+i][c-i] == piece for i in range(4)):
                return True
    return False

def is_terminal(board):
    """Check if the game is over (someone won or board is full)."""
    return check_win(board, PLAYER) or check_win(board, AI) or len(get_valid_columns(board)) == 0

def print_board(board):
    """Display the board in a readable format."""
    symbols = {EMPTY: '⚫', PLAYER: '🔴', AI: '🟡'}
    print()
    print('  '.join(f' {i+1}' for i in range(COLS)))
    print('─' * (COLS * 4 - 1))
    for row in board:
        print('  '.join(f' {symbols[cell]}' for cell in row))
    print('─' * (COLS * 4 - 1))
    print()

# Quick test
board = create_board()
print("Empty board:")
print_board(board)
print(f"Board shape: {board.shape}")
print(f"Valid columns: {get_valid_columns(board)}")


Empty board:

 1   2   3   4   5   6   7
───────────────────────────
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
───────────────────────────

Board shape: (6, 7)
Valid columns: [0, 1, 2, 3, 4, 5, 6]


In [3]:
# ===== HEURISTIC EVALUATION =====

def score_window(window, piece):
    """Score a window of 4 cells from the AI's perspective.

    A 'window' is any 4 consecutive cells (horizontal, vertical, or diagonal).
    We count how many of each type are in the window to estimate its value.
    """
    opp = PLAYER if piece == AI else AI
    p_count = sum(1 for x in window if x == piece)   # Our pieces
    e_count = sum(1 for x in window if x == EMPTY)    # Empty spaces
    o_count = sum(1 for x in window if x == opp)      # Opponent pieces

    if p_count == 4: return 100       # WIN!
    if p_count == 3 and e_count == 1: return 5   # Strong threat
    if p_count == 2 and e_count == 2: return 2   # Developing
    if o_count == 3 and e_count == 1: return -4  # Must block opponent!
    return 0


def score_position(board, piece):
    """Evaluate the entire board position for 'piece'.

    Scans every possible window of 4 in all directions,
    plus gives a bonus for controlling the center column.
    """
    score = 0

    # Center column preference (strategic advantage)
    center_col = list(board[:, COLS // 2])
    score += center_col.count(piece) * 3

    # Score all horizontal windows
    for r in range(ROWS):
        for c in range(COLS - 3):
            window = list(board[r, c:c+4])
            score += score_window(window, piece)

    # Score all vertical windows
    for c in range(COLS):
        for r in range(ROWS - 3):
            window = [board[r+i][c] for i in range(4)]
            score += score_window(window, piece)

    # Score diagonal (↘) windows
    for r in range(ROWS - 3):
        for c in range(COLS - 3):
            window = [board[r+i][c+i] for i in range(4)]
            score += score_window(window, piece)

    # Score diagonal (↙) windows
    for r in range(ROWS - 3):
        for c in range(3, COLS):
            window = [board[r+i][c-i] for i in range(4)]
            score += score_window(window, piece)

    return score


# Demo: score an empty board vs a board with center pieces
demo_board = create_board()
print(f"Empty board score (AI perspective): {score_position(demo_board, AI)}")

demo_board2 = drop_piece(demo_board, 3, AI)  # AI plays center
demo_board2 = drop_piece(demo_board2, 3, AI)  # AI plays center again
print(f"Board with 2 AI pieces in center:  {score_position(demo_board2, AI)}")
print("\n↑ Center control gives a strategic advantage!")


Empty board score (AI perspective): 0
Board with 2 AI pieces in center:  8

↑ Center control gives a strategic advantage!


In [4]:
# ===== MINIMAX WITH ALPHA-BETA PRUNING =====

def minimax(board, depth, alpha, beta, is_maximizing):
    """
    Minimax algorithm with Alpha-Beta pruning.

    Parameters:
        board: current game state
        depth: how many more levels to search (0 = evaluate now)
        alpha: best score the Maximizer (AI) can guarantee so far
        beta:  best score the Minimizer (Player) can guarantee so far
        is_maximizing: True if it's AI's turn (wants HIGH scores)

    Returns:
        (best_column, best_score)

    Key insight:
        - AI is the MAXIMIZER (wants the highest score)
        - Player is the MINIMIZER (wants the lowest score)
        - Alpha-Beta pruning: if we find a branch that's already worse than
          a known option, we skip it (saves HUGE amounts of computation)
    """
    valid_cols = get_valid_columns(board)
    terminal = is_terminal(board)

    # Base cases: game over or depth limit reached
    if depth == 0 or terminal:
        if terminal:
            if check_win(board, AI):     return (None, 100_000)   # AI wins
            if check_win(board, PLAYER): return (None, -100_000)  # Player wins
            return (None, 0)                                       # Draw
        return (None, score_position(board, AI))  # Heuristic evaluation

    if is_maximizing:  # AI's turn — wants to MAXIMIZE
        value = -np.inf
        best_col = valid_cols[np.random.randint(len(valid_cols))]

        for col in valid_cols:
            new_board = drop_piece(board, col, AI)
            _, score = minimax(new_board, depth - 1, alpha, beta, False)

            if score > value:
                value = score
                best_col = col

            alpha = max(alpha, value)
            if alpha >= beta:  # ✂️ PRUNE! Player would never allow this pathh
                break

        return best_col, value

    else:  # Player's turn — wants to MINIMIZE
        value = np.inf
        best_col = valid_cols[np.random.randint(len(valid_cols))]

        for col in valid_cols:
            new_board = drop_piece(board, col, PLAYER)
            _, score = minimax(new_board, depth - 1, alpha, beta, True)

            if score < value:
                value = score
                best_col = col

            beta = min(beta, value)
            if alpha >= beta:  # ✂️ PRUNE! AI would never allow this path
                break

        return best_col, value


def ai_move(board, depth=4):
    """Get the AI's best move at the given search depth.

    Also measures how long the AI takes to 'think'.
    """
    start_time = time.time()
    col, score = minimax(board, depth, -np.inf, np.inf, True)
    elapsed = time.time() - start_time
    return col, score, elapsed


#  let the AI pick a move on an empty board demo
demo_board = create_board()
col, score, elapsed = ai_move(demo_board, depth=4)
print(f"AI's first move: column {col + 1} (score: {score}, took {elapsed:.3f}s)")
print("\n💡 The AI almost always opens in the center — it's the strongest first move!")


AI's first move: column 4 (score: 6, took 0.085s)

💡 The AI almost always opens in the center — it's the strongest first move!


In [5]:
# Difficulty configurations
DIFFICULTIES = {
    '1': {'name': '🟢 Rookie',      'depth': 2, 'blunder': 0.25},
    '2': {'name': '🟡 Tactician',   'depth': 4, 'blunder': 0.08},
    '3': {'name': '🔴 Grandmaster', 'depth': 6, 'blunder': 0.00},
}

# Show search depth comparison
print("How depth affects AI thinking:\n")
test_board = create_board()
test_board = drop_piece(test_board, 3, PLAYER)
test_board = drop_piece(test_board, 3, AI)
test_board = drop_piece(test_board, 4, PLAYER)

for depth in [2, 4, 6]:
    col, score, elapsed = ai_move(test_board, depth=depth)
    print(f"  Depth {depth}: chooses col {col+1}, score={score:>8.0f}, time={elapsed:.4f}s")

print("\n↑ Deeper search = better decisions but more computation time")


How depth affects AI thinking:

  Depth 2: chooses col 3, score=       1, time=0.0094s
  Depth 4: chooses col 3, score=       5, time=0.1650s
  Depth 6: chooses col 3, score=       6, time=4.1201s

↑ Deeper search = better decisions but more computation time


In [9]:
# 🎮 PLAY CONNECT 4 vs AI!
# Run this cell to start a game

import random

def play_game():
    """Main game loop — play Connect 4 against the AI!"""

    print("╔══════════════════════════════════╗")
    print("║     🎮 CONNECT 4 vs AI 🤖       ║")
    print("╠══════════════════════════════════╣")
    print("║  You: 🔴   AI: 🟡               ║")
    print("║  Get 4 in a row to win!          ║")
    print("╚══════════════════════════════════╝")
    print()

    # Choose difficulty
    print("Select difficulty:")
    for key, diff in DIFFICULTIES.items():
        print(f"  [{key}] {diff['name']} (depth {diff['depth']})")
    print()

    while True:
        choice = input("Enter 1, 2, or 3: ").strip()
        if choice in DIFFICULTIES:
            break
        print("Invalid choice. Try again.")

    diff = DIFFICULTIES[choice]
    depth = diff['depth']
    blunder_rate = diff['blunder']
    print(f"\n🎯 Playing against {diff['name']}!\n")

    # AI taunts
    taunts = [
        "Interesting move... 🤔", "Bold strategy!", "I see what you're doing 👀",
        "Not bad!", "Hmm, let me think...", "Is that your best? 😏",
        "Clever!", "I expected that.", "Surprising choice!",
        "You're making this fun!", "Watch this...", "My turn! 🎯",
    ]

    board = create_board()
    game_over = False
    move_count = 0

    while not game_over:
        print_board(board)
        valid = get_valid_columns(board)

        # ---- PLAYER'S TURN ----
        print(f"Your turn 🔴  (valid columns: {[c+1 for c in valid]})")
        while True:
            try:
                col = int(input("Drop in column (1-7): ")) - 1
                if col in valid:
                    break
                print(f"Column {col+1} is full! Choose another.")
            except (ValueError, EOFError):
                print("Enter a number 1-7.")

        board = drop_piece(board, col, PLAYER)
        move_count += 1

        if check_win(board, PLAYER):
            print_board(board)
            print("🎉🎉🎉 YOU WIN! 🎉🎉🎉")
            print(f"Congratulations! You beat {diff['name']} in {move_count} moves!")
            game_over = True
            continue

        if len(get_valid_columns(board)) == 0:
            print_board(board)
            print("🤝 It's a DRAW!")
            game_over = True
            continue

        # ---- AI'S TURN ----
        print(f"\n🤖 AI is thinking", end="", flush=True)

        # Blunder mechanic for easier difficulties
        if blunder_rate > 0 and random.random() < blunder_rate:
            valid_cols = get_valid_columns(board)
            ai_col = random.choice(valid_cols)
            time.sleep(0.5)
            print("...")
        else:
            ai_col, ai_score, elapsed = ai_move(board, depth=depth)
            for _ in range(3):
                time.sleep(0.2)
                print(".", end="", flush=True)
            print()

        board = drop_piece(board, ai_col, AI)
        move_count += 1

        print(f"🤖 AI drops in column {ai_col + 1}  — {random.choice(taunts)}")

        if check_win(board, AI):
            print_board(board)
            print(f"🤖 AI WINS! Better luck next time!")
            print(f"The {diff['name']} beat you in {move_count} moves.")
            game_over = True
            continue

        if len(get_valid_columns(board)) == 0:
            print_board(board)
            print("🤝 It's a DRAW!")
            game_over = True
            continue

    # Play again?
    print()
    again = input("Play again? (y/n): ").strip().lower()
    if again == 'y':
        print("\n" + "="*40 + "\n")
        play_game()

# Start the game!
play_game()


╔══════════════════════════════════╗
║     🎮 CONNECT 4 vs AI 🤖       ║
╠══════════════════════════════════╣
║  You: 🔴   AI: 🟡               ║
║  Get 4 in a row to win!          ║
╚══════════════════════════════════╝

Select difficulty:
  [1] 🟢 Rookie (depth 2)
  [2] 🟡 Tactician (depth 4)
  [3] 🔴 Grandmaster (depth 6)


🎯 Playing against 🟢 Rookie!


 1   2   3   4   5   6   7
───────────────────────────
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
───────────────────────────

Your turn 🔴  (valid columns: [1, 2, 3, 4, 5, 6, 7])

🤖 AI is thinking...
🤖 AI drops in column 4  — Bold strategy!

 1   2   3   4   5   6   7
───────────────────────────
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   ⚫   ⚫   ⚫   ⚫
 ⚫   ⚫   ⚫   🟡   ⚫   ⚫   🔴
───────────────────────────

Your turn 🔴  (valid columns: [1, 2, 3, 4, 

KeyboardInterrupt: Interrupted by user

In [8]:
# 🤖 AI vs AI — watch two AIs battle it out!

def ai_vs_ai(depth_1=2, depth_2=6, pause=0.3):
    """Watch two AIs play against each other.

    Args:
        depth_1: search depth for AI 1 (🔴)
        depth_2: search depth for AI 2 (🟡)
        pause: seconds between moves (for readability)
    """
    print(f"\n🤖 AI Battle: Depth {depth_1} (🔴) vs Depth {depth_2} (🟡)")
    print("=" * 45)

    board = create_board()
    move_count = 0

    while True:
        # AI 1's turn (plays as PLAYER / 🔴)
        col1, score1, t1 = minimax(board, depth_1, -np.inf, np.inf, False)
        # For AI 1 we use is_maximizing=False because it plays as PLAYER (minimizer)
        # Actually let's just use the minimax properly:
        # AI 1 wants to maximize for PLAYER's perspective
        valid = get_valid_columns(board)

        # Simple approach: AI 1 picks best move for PLAYER
        best_col, best_score = None, -np.inf
        for c in valid:
            nb = drop_piece(board, c, PLAYER)
            _, s = minimax(nb, depth_1 - 1, -np.inf, np.inf, True)
            # Lower score is better for PLAYER (minimizer)
            if -s > best_score:
                best_score = -s
                best_col = c

        board = drop_piece(board, best_col, PLAYER)
        move_count += 1

        if IN_NOTEBOOK:
            clear_output(wait=True)
            print(f"🤖 AI Battle: Depth {depth_1} (🔴) vs Depth {depth_2} (🟡)")
            print(f"Move {move_count}: 🔴 (depth {depth_1}) → column {best_col + 1}")
            print_board(board)

        if check_win(board, PLAYER):
            if not IN_NOTEBOOK:
                print_board(board)
            print(f"\n🔴 Depth {depth_1} WINS in {move_count} moves! (Upset!)")
            return
        if len(get_valid_columns(board)) == 0:
            if not IN_NOTEBOOK:
                print_board(board)
            print(f"\n🤝 DRAW after {move_count} moves!")
            return

        time.sleep(pause)

        # AI 2's turn (plays as AI / 🟡)
        col2, score2, t2 = ai_move(board, depth=depth_2)
        board = drop_piece(board, col2, AI)
        move_count += 1

        if IN_NOTEBOOK:
            clear_output(wait=True)
            print(f"🤖 AI Battle: Depth {depth_1} (🔴) vs Depth {depth_2} (🟡)")
            print(f"Move {move_count}: 🟡 (depth {depth_2}) → column {col2 + 1}")
            print_board(board)

        if check_win(board, AI):
            if not IN_NOTEBOOK:
                print_board(board)
            print(f"\n🟡 Depth {depth_2} WINS in {move_count} moves!")
            return
        if len(get_valid_columns(board)) == 0:
            if not IN_NOTEBOOK:
                print_board(board)
            print(f"\n🤝 DRAW after {move_count} moves!")
            return

        time.sleep(pause)

# Watch Rookie (depth 2) vs Grandmaster (depth 6)!
ai_vs_ai(depth_1=2, depth_2=6, pause=0.1)



🤖 AI Battle: Depth 2 (🔴) vs Depth 6 (🟡)


ValueError: not enough values to unpack (expected 3, got 2)